In [165]:
from selenium.webdriver import Firefox
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options

In [183]:
import PIL
from PIL import Image
from pytesseract import image_to_string
import pytesseract
import numpy as np, matplotlib.pyplot as plt, pandas as pd

### Load Firefox and get the Lakport Schedule page

In [167]:
opts = Options()
opts.headless=True
browser = Firefox(options=opts)
browser.get('http://lakport.nic.in/ship_online_programme.aspx')

### Select All Passenger Ships option

In [168]:
#Select All Passenger Ships in drop down selection
ship_selector=Select(browser.find_element_by_id("ContentPlaceHolder1_ship_dlist"))
ship_selector.select_by_visible_text("All Passenger Ships")
ship_selector=Select(browser.find_element_by_id("ContentPlaceHolder1_ship_dlist"))
print(ship_selector.all_selected_options[0].text)

All Passenger Ships


### Defining all required methods to extract and solve captcha

In [184]:
def extract_capcha(browser):
    """Extract the captcha from browser and return
    """
    img=browser.find_element_by_css_selector('img')
    import os, random
    os.makedirs('captcha', exist_ok=True) # Serialize the captchas
    captchas=[int(filename.split('.')[0].split('_')[1]) for filename 
              in os.listdir('captcha') if 'cap' in filename]
    # New name = old name + 1
    if len(captchas)==0: captchas=[-1]
    cap_name = 'captcha/cap_'+ str(max(captchas)+1) +'.png'
    img.screenshot(cap_name)
    return Image.open(cap_name)

def display_captcha(browser):
    plt.imshow(extract_capcha(browser))
    print("OCR Prediction: ", image_to_string(extract_capcha(browser)))
    plt.show()

In [185]:
def solve_captcha(browser):
    """Solve the captcha using teserract and press Enter/Return
    """
    captcha_inp=browser.find_element_by_id('ContentPlaceHolder1_txtimage')
    captcha_inp.clear()
    captcha_inp.send_keys(image_to_string(extract_capcha(browser)))
    captcha_inp.send_keys(Keys.RETURN)

In [186]:
def solve_captcha_again(browser, display=True):
    """Get New Captcha and solve it again
    """
    # Button to refresh the captcha
    refresh_button=browser.find_element_by_id('ContentPlaceHolder1_ImageButton')
    refresh_button.click()
    if display: display_captcha(browser)
    solve_captcha(browser)

In [187]:
def is_captcha_valid(browser):
    """Check if the page contains the required table
    If solve_captcha worked, it will or else need to solve new captcha
    """
    from selenium.common.exceptions import NoSuchElementException
    try: 
        _ = browser.find_element_by_id('ContentPlaceHolder1_progallships_dgrid')
        #Schedule has been found in page without error
        return True
    except NoSuchElementException: return False    

### Keep Solving Captcha until correct

In [182]:
# Try new captchas untill correct prediction ;)
for i in range(100):
    if is_captcha_valid(browser): break #break out of for loop
    solve_captcha_again(browser, display=True)
    from time import sleep
    sleep(1)

### Save the page to html

In [177]:
with open('../IE504 Assignment/Lakport_schedule.html', 'w') as f:
    f.write(browser.page_source)

### Showing the read timetable from saved html

In [181]:
pd.read_html('../IE504 Assignment/Lakport_schedule.html', header=0)[0]

,Date,Amindivi,Arabian Sea,Bharat Seema,Corals,Kavaratti,Lagoons,Lakshadweep Sea,Minicoy
0,05/03/2019,NaN,NaN,NaN,NaN,Kochi - 20:00,NaN,NaN,NaN
1,06/03/2019,NaN,NaN,NaN,NaN,Kalpeni - 09:00Androth - 13:00Kalpeni - 17:00,NaN,NaN,NaN
2,07/03/2019,Mangalore - 11:00,Kochi - 13:30,NaN,NaN,Minicoy - 15:30,Kochi - 14:00,NaN,NaN
3,08/03/2019,Kiltan - 10:00Chetlat - 15:00,Minicoy - 13:00,NaN,NaN,Kavaratti - 09:00Agatti - 15:00Kavaratti - 18:00,Kavaratti - 11:00Agatti - 17:00,NaN,NaN
4,09/03/2019,Mangalore - 07:00,Kochi - 10:00,NaN,Kochi - 17:00,Kochi - 10:00,Amini - 10:00Kadmat - 17:00,Kochi - 13:00,Beypore - 13:00
5,10/03/2019,Mangalore - 13:30,Kochi - 13:00,NaN,Androth - 10:00Kalpeni - 17:00,NaN,Kochi - 10:00,Kavaratti - 09:00Amini - 13:00Kadmat - 17:00,Chetlat - 10:00Kiltan - 16:00
6,11/03/2019,Amini - 09:00Kadmat - 13:00Kavaratti - 17:00,Minicoy - 13:00,NaN,Kochi - 10:00,Kochi - 13:00,NaN,Kochi - 10:00,Beypore - 08:00
7,12/03/2019,Mangalore - 09:00,Kochi - 10:00,NaN,NaN,Kavaratti - 09:00Agatti - 14:00Kavaratti - 18:00,NaN,NaN,Beypore - 17:00
8,13/03/2019,NaN,NaN,NaN,Kochi - 13:00,Minicoy - 16:00,NaN,Kochi - 13:00,Androth - 10:00Kalpeni - 17:00
9,14/03/2019,NaN,NaN,NaN,Kavaratti - 10:00Agatti - 17:00,Kalpeni - 09:00Androth - 13:00Kalpeni - 18:00,Kochi - 12:00,Kadmat - 09:00Amini - 12:00Kavaratti - 17:00,Beypore - 07:00
